### Setup

In [1]:
import os
from pathlib import Path

import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
import numpy as np
# import seaborn as sns
from torch.utils.data import TensorDataset, DataLoader
from torch.nn import Sequential, Linear, ReLU, LeakyReLU, Sigmoid, Dropout
from train import _compute_metrics, train_epoch, eval_epoch, train_model

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# import matplotlib.pyplot as plt

from train import train_model
import pickle


In [2]:
project_path = Path(Path.cwd()).parent.parent

In [3]:
SEED = 7777

### Load Data

In [4]:
cancer_detection_path = project_path / "data/inputs/Lung Cancer Dataset.csv"
df_detection = pd.read_csv(cancer_detection_path)

df_detection.columns = [x for x in df_detection.columns.str.lower().str.replace(" ", "_")]

df_detection.head(10)

,age,gender,smoking,finger_discoloration,mental_stress,exposure_to_pollution,long_term_illness,energy_level,immune_weakness,breathing_issue,alcohol_consumption,throat_discomfort,oxygen_saturation,chest_tightness,family_history,smoking_family_history,stress_immune,pulmonary_disease
0,68,1,1,1,1,1,0,57.831178,0,0,1,1,95.977287,1,0,0,0,NO
1,81,1,1,0,0,1,1,47.694835,1,1,0,1,97.184483,0,0,0,0,YES
2,58,1,1,0,0,0,0,59.577435,0,1,1,0,94.974939,0,0,0,0,NO
3,44,0,1,0,1,1,0,59.785767,0,1,0,1,95.187900,0,0,0,0,YES
4,72,0,1,1,1,1,1,59.733941,0,1,0,1,93.503008,0,0,0,0,YES
5,37,1,1,1,1,1,1,57.684285,0,1,1,1,94.057151,1,0,0,0,YES
6,50,0,1,1,1,0,1,52.647022,1,1,1,0,96.773598,0,0,0,1,NO
7,68,0,1,1,1,0,1,53.306451,0,0,0,1,95.019018,0,0,0,0,NO
8,48,0,1,1,0,1,1,64.272789,1,1,0,1,98.539379,1,0,0,0,YES
9,52,0,0,0,1,1,1,58.319319,0,1,0,1,96.055097,0,0,0,0,NO


In [5]:
df_detection.shape

(5000, 18)

### Preprocessing

In [6]:
# Convert label column to numerical values
label_map_dict = {
    'NO': 0,
    'YES': 1
}

df_detection['pulmonary_disease'] = df_detection['pulmonary_disease'].map(label_map_dict)

In [7]:
# Convert binary columns to categorical
binary_columns = [
    'gender',
    'smoking',
    'finger_discoloration',
    'mental_stress',
    'exposure_to_pollution',
    'long_term_illness',
    'immune_weakness',
    'breathing_issue',
    'alcohol_consumption',
    'throat_discomfort',
    'chest_tightness',
    'family_history',
    'smoking_family_history',
    'stress_immune',
    'pulmonary_disease'
]

df_detection[binary_columns] = df_detection[binary_columns].astype('category')

In [8]:
X = df_detection.drop(columns=['pulmonary_disease']).values
y = df_detection['pulmonary_disease'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, random_state=SEED, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.15, random_state=SEED, stratify=y_train)

In [9]:
print("Train set shape:", X_train.shape)
print("Validation set shape:", X_val.shape)
print("Test set shape:", X_test.shape)

Train set shape: (3612, 17)
Validation set shape: (638, 17)
Test set shape: (750, 17)


In [10]:
X_train.mean(axis=0), X_train.std(axis=0)

(array([57.37015504,  0.49612403,  0.66196013,  0.60354374,  0.54512735,
         0.51522702,  0.43992248, 54.99256295,  0.39451827,  0.79983389,
         0.35022148,  0.69988926, 94.9913959 ,  0.6013289 ,  0.303433  ,
         0.20265781,  0.21179402]),
 array([15.83079571,  0.49998498,  0.47304219,  0.48916121,  0.49795936,
         0.49976808,  0.49637757,  7.84740972,  0.48874697,  0.40012453,
         0.4770392 ,  0.45830589,  1.49321387,  0.48962481,  0.4597406 ,
         0.40197963,  0.40857963]))

In [11]:
with open("ga_results.pkl", "rb") as f:
    ga_results = pickle.load(f)

In [12]:
best_feature_set = ga_results['best_individual']
best_feature_mask = np.array(best_feature_set, dtype=bool)
print(best_feature_set)
print(best_feature_mask)

[1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 1]
[ True False  True  True False  True False  True  True  True False  True
 False False  True  True  True]


In [13]:
X_train = X_train[:,best_feature_mask]
X_val = X_val[:,best_feature_mask]
X_test = X_test[:,best_feature_mask]

In [14]:
scaler = StandardScaler()

X_train = scaler.fit_transform(X_train)
X_val = scaler.transform(X_val)
X_test = scaler.transform(X_test)

### GA-FS performance

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device = torch.device(device)
epochs = 300

In [ ]:
train_dataset = TensorDataset(torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long))
val_dataset = TensorDataset(torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long))
test_dataset = TensorDataset(torch.tensor(X_test, dtype=torch.float32), torch.tensor(y_test, dtype=torch.long))

mlp = Sequential(
    Linear(11, 64),
    ReLU(),
    Dropout(0.4),
    Linear(64, 32),
    ReLU(),
    Dropout(0.4),
    Linear(32, 16),
    ReLU(),
    Dropout(0.1),
    Linear(16, 2)
)

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(mlp.parameters(), lr=0.0005, weight_decay=1e-3)



train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, pin_memory=True)
val_loader = DataLoader(val_dataset, batch_size=128, shuffle=False, pin_memory=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False, pin_memory=True)

In [ ]:
results = train_model(mlp, train_loader, val_loader, criterion, optimizer, device)

Starting training for 300 epochs with patience=75
Epoch   1/300 | Train: Loss 0.6826 Acc 57.12% F1 0.5923 AUROC 0.6444 | Val: Loss 0.6418 Acc 81.03% F1 0.7218 AUROC 0.8864
Epoch  10/300 | Train: Loss 0.3474 Acc 88.21% F1 0.8551 AUROC 0.9197 | Val: Loss 0.2708 Acc 91.69% F1 0.8983 AUROC 0.9445
Epoch  20/300 | Train: Loss 0.3252 Acc 89.81% F1 0.8742 AUROC 0.9233 | Val: Loss 0.2568 Acc 92.48% F1 0.9077 AUROC 0.9451
Epoch  30/300 | Train: Loss 0.3117 Acc 89.95% F1 0.8759 AUROC 0.9291 | Val: Loss 0.2474 Acc 92.79% F1 0.9112 AUROC 0.9453
Epoch  40/300 | Train: Loss 0.3000 Acc 90.31% F1 0.8807 AUROC 0.9321 | Val: Loss 0.2415 Acc 93.10% F1 0.9141 AUROC 0.9472
Epoch  50/300 | Train: Loss 0.3016 Acc 90.42% F1 0.8813 AUROC 0.9279 | Val: Loss 0.2381 Acc 93.10% F1 0.9151 AUROC 0.9463
Epoch  60/300 | Train: Loss 0.2959 Acc 91.00% F1 0.8892 AUROC 0.9271 | Val: Loss 0.2346 Acc 93.57% F1 0.9204 AUROC 0.9468
Epoch  70/300 | Train: Loss 0.2940 Acc 90.70% F1 0.8851 AUROC 0.9285 | Val: Loss 0.2345 Acc 93.5

In [ ]:
results

{'history': {'train_loss': [0.6826017606562026,
   0.5845653975788805,
   0.43680377916772234,
   0.38684230247473267,
   0.37923739137641616,
   0.37093565887787017,
   0.3531559946331075,
   0.35767152423610454,
   0.34649571172422744,
   0.3474013962156881,
   0.3417155562445175,
   0.3404845670608191,
   0.34186668814482746,
   0.34758039493761456,
   0.33622041202726816,
   0.3282901802464313,
   0.33028922600471566,
   0.3237342142385502,
   0.325867342674983,
   0.325176298684324,
   0.3262396673295453,
   0.32826001834895785,
   0.31625809862896187,
   0.3206654550427218,
   0.3171541178774332,
   0.3099159032502444,
   0.31297693517913056,
   0.3114269238505781,
   0.31101381501486136,
   0.3116893006255592,
   0.3120533089470098,
   0.3112445581592991,
   0.30589934716523026,
   0.30903150015825187,
   0.3121651195542493,
   0.30856863730241557,
   0.31095457813008415,
   0.3095104293041715,
   0.30186246125933075,
   0.30002068364342976,
   0.3070234376462723,
   0.306702793

In [ ]:
# Load best model and evaluate on test set
mlp.load_state_dict(torch.load('best_model.pth'))
test_loss, test_acc, test_labels, test_preds, test_probs = eval_epoch(mlp, test_loader, criterion, device)
test_precision, test_recall, test_f1, test_auroc = _compute_metrics(test_labels, test_preds, test_probs)

print(f"\nRESULTADOS DEL CONJUNTO DE PRUEBA (usando el mejor modelo):")
print(f"Pérdida: {test_loss:.4f}, Accuracy: {test_acc:.2f}%, Precision: {test_precision:.4f}, "
      f"Recall: {test_recall:.4f}, F1: {test_f1:.4f}, AUROC: {test_auroc:.4f}")


RESULTADOS DEL CONJUNTO DE PRUEBA (usando el mejor modelo):
Pérdida: 0.3057, Accuracy: 90.27%, Precision: 0.8722, Recall: 0.8922, F1: 0.8821, AUROC: 0.9118


### PSO

In [17]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from pso_hyperparameter_tuning import run_pso_hyperparameter_tuning

# Your data (11 features, already scaled)
pso_results = run_pso_hyperparameter_tuning(
    X_train_scaled=X_train,
    y_train=y_train,
    X_val_scaled=X_val,
    y_val=y_val,
    device=device,
    n_trials=100,
    n_particles=20,
    epochs_per_trial=250,
    patience=50,
    inertia=0.7,
    cognitive=1.4,
    social=1.4,
    verbose=True,
    seed=42
)

# Get optimized model
best_model = pso_results['final_model']
best_params = pso_results['best_params']

[I 2025-10-13 19:16:10,723] A new study created in memory with name: no-name-1204d39b-7896-4ad9-8d81-fe1897d1284c


Starting PSO Hyperparameter Optimization
Fixed Architecture: 3 hidden layers, ReLU activation, Adam optimizer
Input features: 11
Training samples: 3612
Validation samples: 638
PSO Settings: 20 particles, 100 trials
PSO Coefficients: inertia=0.7, cognitive=1.4, social=1.4


Best trial: 0. Best value: 0.224977:   1%|          | 1/100 [00:30<49:43, 30.14s/it]

[I 2025-10-13 19:16:40,852] Trial 0 finished with value: 0.22497667563746343 and parameters: {'use_batch_norm': False, 'learning_rate': 0.001570297088405539, 'weight_decay': 0.0002481040974867811, 'batch_size': 32, 'hidden_size_0': 64, 'hidden_size_1': 16, 'hidden_size_2': 16, 'dropout_rate_0': 0.5330880728874676, 'dropout_rate_1': 0.40055750587160444, 'dropout_rate_2': 0.4540362888980227}. Best is trial 0 with value: 0.22497667563746343.
Trial 0/100 | Best value: 0.2250 | Best trial: 0


Best trial: 1. Best value: 0.22448:   2%|▏         | 2/100 [00:50<39:36, 24.25s/it] 

...

[I 2025-10-13 20:44:10,416] Trial 97 finished with value: 0.22763592908655214 and parameters: {'use_batch_norm': True, 'learning_rate': 0.0007876558269092243, 'weight_decay': 2.337116366526096e-05, 'batch_size': 32, 'hidden_size_0': 32, 'hidden_size_1': 32, 'hidden_size_2': 16, 'dropout_rate_0': 0.3491493082326493, 'dropout_rate_1': 0.30339732200622066, 'dropout_rate_2': 0.16755239093053553}. Best is trial 69 with value: 0.2181567822037072.


Best trial: 69. Best value: 0.218157:  99%|█████████▉| 99/100 [1:28:17<00:47, 47.29s/it]

[I 2025-10-13 20:44:28,343] Trial 98 finished with value: 0.22158491382591403 and parameters: {'use_batch_norm': False, 'learning_rate': 0.006655976054597098, 'weight_decay': 0.00040339298096726296, 'batch_size': 112, 'hidden_size_0': 32, 'hidden_size_1': 16, 'hidden_size_2': 16, 'dropout_rate_0': 0.35232746887981947, 'dropout_rate_1': 0.4486093314806624, 'dropout_rate_2': 0.18453145845842836}. Best is trial 69 with value: 0.2181567822037072.


Best trial: 99. Best value: 0.218041: 100%|██████████| 100/100 [1:29:01<00:00, 53.42s/it]


[I 2025-10-13 20:45:12,431] Trial 99 finished with value: 0.21804061915059822 and parameters: {'use_batch_norm': False, 'learning_rate': 0.0019261262846184653, 'weight_decay': 0.00011834720359209613, 'batch_size': 80, 'hidden_size_0': 32, 'hidden_size_1': 32, 'hidden_size_2': 32, 'dropout_rate_0': 0.4653610767143221, 'dropout_rate_1': 0.38465019114405796, 'dropout_rate_2': 0.38654987245975936}. Best is trial 99 with value: 0.21804061915059822.

PSO Optimization Complete!
Best validation loss: 0.2180
Best trial: 99

Best Hyperparameters:
  Architecture: 3 layers with ReLU activation
  Layer 1: 32 neurons, dropout=0.465
  Layer 2: 32 neurons, dropout=0.385
  Layer 3: 32 neurons, dropout=0.387
  Learning rate: 0.001926
  Weight decay: 0.000118
  Batch size: 80
  Batch normalization: False

Training final model with best hyperparameters...
Starting training for 300 epochs with patience=50
Epoch   1/300 | Train: Loss 0.6208 Acc 66.06% F1 0.3732 AUROC 0.7156 | Val: Loss 0.4326 Acc 86.36% F1 

In [19]:
pso_results

{'study': <optuna.study.study.Study at 0x1f1c5a7c830>,
 'best_params': {'use_batch_norm': False,
  'learning_rate': 0.0019261262846184653,
  'weight_decay': 0.00011834720359209613,
  'batch_size': 80,
  'hidden_size_0': 32,
  'hidden_size_1': 32,
  'hidden_size_2': 32,
  'dropout_rate_0': 0.4653610767143221,
  'dropout_rate_1': 0.38465019114405796,
  'dropout_rate_2': 0.38654987245975936},
 'best_value': 0.21804061915059822,
 'best_trial': FrozenTrial(number=99, state=1, values=[0.21804061915059822], datetime_start=datetime.datetime(2025, 10, 13, 20, 44, 28, 347763), datetime_complete=datetime.datetime(2025, 10, 13, 20, 45, 12, 431313), params={'use_batch_norm': False, 'learning_rate': 0.0019261262846184653, 'weight_decay': 0.00011834720359209613, 'batch_size': 80, 'hidden_size_0': 32, 'hidden_size_1': 32, 'hidden_size_2': 32, 'dropout_rate_0': 0.4653610767143221, 'dropout_rate_1': 0.38465019114405796, 'dropout_rate_2': 0.38654987245975936}, user_attrs={}, system_attrs={}, intermediate

In [20]:
import pickle
with open("pso_results.pkl", "wb") as f:
    pickle.dump(pso_results, f)

### PSO 2

In [15]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [ ]:
from pso_hyperparameter_tuning import run_pso_hyperparameter_tuning

# Your data (11 features, already scaled)
pso_results = run_pso_hyperparameter_tuning(
    X_train_scaled=X_train,  # Shape: (n_samples, 11)
    y_train=y_train,
    X_val_scaled=X_val,
    y_val=y_val,
    device=device,
    n_trials=100,
    n_particles=25,
    epochs_per_trial=300,
    patience=75,
    inertia=0.8,
    cognitive=1.4,
    social=1.7,
    verbose=True,
    seed=42
)

# Get optimized model
best_model = pso_results['final_model']
best_params = pso_results['best_params']

[I 2025-10-13 20:56:47,560] A new study created in memory with name: no-name-7fa3cb72-5254-4e2f-b31c-319a6551784b


Starting PSO Hyperparameter Optimization
Fixed Architecture: 3 hidden layers, ReLU activation, Adam optimizer
Input features: 11
Training samples: 3612
Validation samples: 638
PSO Settings: 25 particles, 100 trials
PSO Coefficients: inertia=0.8, cognitive=1.4, social=1.7


Best trial: 0. Best value: 0.222096:   1%|          | 1/100 [00:36<59:28, 36.04s/it]

[I 2025-10-13 20:57:23,606] Trial 0 finished with value: 0.2220956461977062 and parameters: {'use_batch_norm': False, 'learning_rate': 0.0009454306819536165, 'weight_decay': 0.0002481040974867811, 'batch_size': 48, 'hidden_size_0': 64, 'hidden_size_1': 16, 'hidden_size_2': 16, 'dropout_rate_0': 0.5330880728874676, 'dropout_rate_1': 0.40055750587160444, 'dropout_rate_2': 0.2416145155592091}. Best is trial 0 with value: 0.2220956461977062.
Trial 0/100 | Best value: 0.2221 | Best trial: 0


Best trial: 0. Best value: 0.222096:   2%|▏         | 2/100 [00:56<43:33, 26.67s/it]

...

[I 2025-10-13 21:49:56,201] Trial 97 finished with value: 0.22741331837394022 and parameters: {'use_batch_norm': False, 'learning_rate': 0.00070480486917522, 'weight_decay': 1e-06, 'batch_size': 80, 'hidden_size_0': 160, 'hidden_size_1': 16, 'hidden_size_2': 16, 'dropout_rate_0': 0.5451805905717804, 'dropout_rate_1': 0.6, 'dropout_rate_2': 0.13252338786897827}. Best is trial 59 with value: 0.21596265567881187.


Best trial: 59. Best value: 0.215963:  99%|█████████▉| 99/100 [53:29<00:31, 31.73s/it]

[I 2025-10-13 21:50:16,647] Trial 98 finished with value: 0.22047626023942773 and parameters: {'use_batch_norm': True, 'learning_rate': 0.003660003292622836, 'weight_decay': 1e-06, 'batch_size': 112, 'hidden_size_0': 32, 'hidden_size_1': 32, 'hidden_size_2': 32, 'dropout_rate_0': 0.6, 'dropout_rate_1': 0.5670809637756279, 'dropout_rate_2': 0.14925608122108253}. Best is trial 59 with value: 0.21596265567881187.


Best trial: 59. Best value: 0.215963: 100%|██████████| 100/100 [53:51<00:00, 32.31s/it]


[I 2025-10-13 21:50:38,560] Trial 99 finished with value: 0.2208151582359894 and parameters: {'use_batch_norm': True, 'learning_rate': 0.003291593520634592, 'weight_decay': 1e-06, 'batch_size': 96, 'hidden_size_0': 32, 'hidden_size_1': 16, 'hidden_size_2': 16, 'dropout_rate_0': 0.6, 'dropout_rate_1': 0.46605451817539856, 'dropout_rate_2': 0.23749106866426367}. Best is trial 59 with value: 0.21596265567881187.

PSO Optimization Complete!
Best validation loss: 0.2160
Best trial: 59

Best Hyperparameters:
  Architecture: 3 layers with ReLU activation
  Layer 1: 32 neurons, dropout=0.600
  Layer 2: 16 neurons, dropout=0.509
  Layer 3: 16 neurons, dropout=0.129
  Learning rate: 0.004355
  Weight decay: 0.000001
  Batch size: 96
  Batch normalization: True

Training final model with best hyperparameters...
Starting training for 300 epochs with patience=50
Epoch   1/300 | Train: Loss 0.6159 Acc 65.89% F1 0.5265 AUROC 0.7072 | Val: Loss 0.4514 Acc 85.11% F1 0.8342 AUROC 0.9131
Epoch  10/300 | 

[I 2025-10-13 21:29:19,328] Trial 50 finished with value: 0.23401889564662143 and parameters: 
`{'use_batch_norm': False, 'learning_rate': 0.0005995071316732543, 'weight_decay': 0.004937699977007833, 'batch_size': 96, 'hidden_size_0': 64, 'hidden_size_1': 16, 'hidden_size_2': 16, 'dropout_rate_0': 0.48246507786460235, 'dropout_rate_1': 0.5361848711639932, 'dropout_rate_2': 0.15099363078453903}`

Best is trial 34 with value: 0.22146023387165278.


In [18]:
import pickle
with open("pso_results_2.pkl", "wb") as f:
    pickle.dump(pso_results, f)